In [4]:
#load packages
import os
import time
import numpy as np
#from numpy.lib.stride_tricks import as_strided
import scipy
import scipy.signal

import pandas as pd
import sklearn
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.model_selection import cross_val_score,RandomizedSearchCV
from sklearn.metrics import confusion_matrix
import itertools
from itertools import combinations
# import xlrd
# import xgboost as xgb
# import lightgbm as lgb
# import tensorflow as tf
# from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from tqdm import tqdm
import subprocess
#print(os.getcwd())
from pack import peakutils#(Package from MIT lab)
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

ModuleNotFoundError: No module named 'pack'

In [2]:
#Define load data
def load_data(start,end):
    df = None
    df_res = None
    df_edar = None
    df_edal = None
    
    for i in range (start,end):
        link = 'F:\\UCL\\DT\\stress_data\\BVP_RES_DATA'+ '\\'+str(i)+'.xlsx'
        locals()['data'+str(i)] = pd.read_excel(link)
        #set labels
        data = locals()['data'+str(i)]
        locals()['re_1_'+str(i)] = pd.DataFrame(data.iloc[1:,3]).T
        locals()['re_1_'+str(i)]['Label'] = 'relax'
        locals()['math_e_'+str(i)] = pd.DataFrame(data.iloc[1:,6]).T
        locals()['math_e_'+str(i)]['Label']='math_easy'
        locals()['re_3_'+str(i)] = pd.DataFrame(data.iloc[1:,9]).T
        locals()['re_3_'+str(i)]['Label'] = 'relax'
        locals()['math_h_'+str(i)] = pd.DataFrame(data.iloc[1:,12]).T
        locals()['math_h_'+str(i)]['Label'] = 'math_hard'
        locals()['re_5_'+str(i)] = pd.DataFrame(data.iloc[1:,15]).T
        locals()['re_5_'+str(i)]['Label'] = 'relax'
        #re_math_e1.index=['']
        df = pd.concat((df,locals()['re_1_'+str(i)],locals()['math_e_'+str(i)],locals()['re_3_'+str(i)],locals()['math_h_'+str(i)],
                         locals()['re_5_'+str(i)]),axis=0,ignore_index=True)   
    #Respiration
        locals()['respi_1_'+str(i)] = pd.DataFrame(data.iloc[1:,4]).T
        locals()['respi_1_'+str(i)]['Label'] = 'relax'
        locals()['respi_me_'+str(i)] = pd.DataFrame(data.iloc[1:,7]).T
        locals()['respi_me_'+str(i)]['Label']='math_easy'
        locals()['respi_3_'+str(i)] = pd.DataFrame(data.iloc[1:,10]).T
        locals()['respi_3_'+str(i)]['Label'] = 'relax'
        locals()['respi_mh_'+str(i)] = pd.DataFrame(data.iloc[1:,13]).T
        locals()['respi_mh_'+str(i)]['Label'] = 'math_hard'
        locals()['respi_5_'+str(i)] = pd.DataFrame(data.iloc[1:,16]).T
        locals()['respi_5_'+str(i)]['Label'] = 'relax'
        df_res = pd.concat((df_res,locals()['respi_1_'+str(i)],locals()['respi_me_'+str(i)],locals()['respi_3_'+str(i)],
                             locals()['respi_mh_'+str(i)],locals()['respi_5_'+str(i)]),axis=0,ignore_index=True)  

    #Right EDA
        locals()['EDA_R_1_'+str(i)] = pd.DataFrame(data.iloc[1:,19]).T
        locals()['EDA_R_1_'+str(i)]['Label'] = 'relax'
        locals()['EDA_R_me_'+str(i)] = pd.DataFrame(data.iloc[1:,21]).T
        locals()['EDA_R_me_'+str(i)]['Label']='math_easy'
        locals()['EDA_R_3_'+str(i)] = pd.DataFrame(data.iloc[1:,23]).T
        locals()['EDA_R_3_'+str(i)]['Label'] = 'relax'
        locals()['EDA_R_mh_'+str(i)] = pd.DataFrame(data.iloc[1:,25]).T
        locals()['EDA_R_mh_'+str(i)]['Label'] = 'math_hard'
        locals()['EDA_R_5_'+str(i)] = pd.DataFrame(data.iloc[1:,27]).T
        locals()['EDA_R_5_'+str(i)]['Label'] = 'relax'
        df_edar = pd.concat((df_edar,locals()['EDA_R_1_'+str(i)],locals()['EDA_R_me_'+str(i)],locals()['EDA_R_3_'+str(i)],
                              locals()['EDA_R_mh_'+str(i)],locals()['EDA_R_5_'+str(i)]),axis=0,ignore_index=True)
    #Left EDA
        locals()['EDA_L_1_'+str(i)] = pd.DataFrame(data.iloc[1:,29]).T
        locals()['EDA_L_1_'+str(i)]['Label'] = 'relax'
        locals()['EDA_L_me_'+str(i)] = pd.DataFrame(data.iloc[1:,31]).T
        locals()['EDA_L_me_'+str(i)]['Label']='math_easy'
        locals()['EDA_L_3_'+str(i)] = pd.DataFrame(data.iloc[1:,33]).T
        locals()['EDA_L_3_'+str(i)]['Label'] = 'relax'
        locals()['EDA_L_mh_'+str(i)] = pd.DataFrame(data.iloc[1:,35]).T
        locals()['EDA_L_mh_'+str(i)]['Label'] = 'math_hard'
        locals()['EDA_L_5_'+str(i)] = pd.DataFrame(data.iloc[1:,37]).T
        locals()['EDA_L_5_'+str(i)]['Label'] = 'relax'
        df_edal = pd.concat((df_edal,locals()['EDA_L_1_'+str(i)],locals()['EDA_L_me_'+str(i)],locals()['EDA_L_3_'+str(i)],
                              locals()['EDA_L_mh_'+str(i)],locals()['EDA_L_5_'+str(i)]),axis=0,ignore_index=True)
        
    return df,df_res,df_edar,df_edal

In [3]:
#Load data
start = time.clock()

df1_BVP,df1_res,df1_edar,df1_edal = load_data(1,7)
df2_BVP,df2_res,df2_edar,df2_edal = load_data(8,11)
df3_BVP,df3_res,df3_edar,df3_edal = load_data(16,24)

df_BVP = pd.concat((df1_BVP,df2_BVP,df3_BVP),axis=0,ignore_index=True)
df_Res = pd.concat((df1_res,df2_res,df3_res),axis=0,ignore_index=True)
df_EDAR = pd.concat((df1_edar,df2_edar,df3_edar),axis=0,ignore_index=True)
df_EDAL = pd.concat((df1_edal,df2_edal,df3_edal),axis=0,ignore_index=True)

elapsed = (time.clock() - start)
print("Time used:",elapsed)

Time used: 2349.047635301568


In [6]:
# df_RES_data =df_Res.iloc[:,0:71609]
# df_RES_label = df_Res.iloc[:,-1]
# df_RES = pd.concat((df_RES_data,df_RES_label),axis=1)
# df_RES.to_csv("F:\\UCL\\DT\\res_concat.csv")

In [4]:
df_BVP.head()
df_Res.head()
df_EDAR.head()
df_EDAL.head()

,1,2,3,4,5,6,7,8,9,10,...,84990,84991,84992,84993,84994,84995,84996,84997,84998,Label
0,0.011527,0.012808,0.012808,0.011527,0.014089,0.012808,0.012808,0.012808,0.012808,0.010246,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,relax
1,0.012808,0.012808,0.014089,0.012808,0.01665,0.012808,0.015369,0.014089,0.015369,0.012808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,math_easy
2,0.01665,0.019212,0.015369,0.01665,0.01665,0.01665,0.01665,0.017931,0.01665,0.01665,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,relax
3,0.024335,0.025616,0.024335,0.015369,0.021773,0.021773,0.024335,0.024335,0.025616,0.025616,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,math_hard
4,0.023054,0.030739,0.033301,0.037143,0.035862,0.03202,0.037143,0.034581,0.023054,0.043547,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,relax


In [8]:
#find the minimum length of BVP and respiration
start = time.clock()
zero_list = []
for indexs in df_BVP.index:
    for  i in range(len(df_BVP.loc[indexs].values)):
        if(df_BVP.loc[indexs].values[i] == 0):
            #print(indexs,i)
            zero_list.append(i)
            break
min_length1 = np.min(zero_list)
print(min_length1)

elapsed = (time.clock() - start)
print("Time used:",elapsed)

F:\Anocanda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


71609


F:\Anocanda\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  del sys.path[0]


Time used: 622.4636640429999


In [11]:
#find the minimum length of edar and edal
start = time.clock()
zero_list = []

for indexs in df_EDAR.index:
    for  i in range(len(df_EDAR.loc[indexs].values)):
        if(df_EDAR.loc[indexs].values[i] ==0):
            #print(indexs,i)
            zero_list.append(i)
            break
min_length2 = np.min(zero_list)
print(min_length2)

elapsed = (time.clock() - start)
print("Time used:",elapsed)

F:\Anocanda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


1137


F:\Anocanda\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


Time used: 9.669871403999423


In [60]:
# train_data=pd.read_csv("F:\\UCL\\DT\\data.csv",sep=",")
# label_data=pd.read_csv("F:\\UCL\\DT\\test.csv",sep=",")

In [5]:
#process to the same length
df_BVP_data = df_BVP.iloc[:,0:71609]
df_BVP_label = df_BVP.iloc[:,-1]
df_BVP = pd.concat((df_BVP_data,df_BVP_label),axis=1)

df_RES_data =df_Res.iloc[:,0:71609]
df_RES_label = df_Res.iloc[:,-1]
df_RES = pd.concat((df_RES_data,df_RES_label),axis=1)

df_EDAR_data = df_EDAR.iloc[:,0:1137]
df_EDAR_label = df_EDAR.iloc[:,-1]
df_EDAR = pd.concat((df_EDAR_data,df_EDAR_label),axis=1)

df_EDAL_data = df_EDAL.iloc[:,0:1137]
df_EDAL_label = df_EDAL.iloc[:,-1]
df_EDAL = pd.concat((df_EDAL_data,df_EDAL_label),axis=1)

df_BVP.head()
df_RES.head()
df_EDAR.head()
df_EDAL.head()

,1,2,3,4,5,6,7,8,9,10,...,1129,1130,1131,1132,1133,1134,1135,1136,1137,Label
0,0.011527,0.012808,0.012808,0.011527,0.014089,0.012808,0.012808,0.012808,0.012808,0.010246,...,0.012808,0.012808,0.012808,0.011527,0.014089,0.012808,0.012808,0.012808,0.011527,relax
1,0.012808,0.012808,0.014089,0.012808,0.01665,0.012808,0.015369,0.014089,0.015369,0.012808,...,0.023054,0.029458,0.025616,0.024335,0.025616,0.025616,0.026897,0.026897,0.023054,math_easy
2,0.01665,0.019212,0.015369,0.01665,0.01665,0.01665,0.01665,0.017931,0.01665,0.01665,...,0.014089,0.015369,0.014089,0.012808,0.015369,0.014089,0.015369,0.015369,0.014089,relax
3,0.024335,0.025616,0.024335,0.015369,0.021773,0.021773,0.024335,0.024335,0.025616,0.025616,...,0.015369,0.015369,0.014089,0.014089,0.014089,0.012808,0.014089,0.012808,0.014089,math_hard
4,0.023054,0.030739,0.033301,0.037143,0.035862,0.03202,0.037143,0.034581,0.023054,0.043547,...,0.01665,0.01665,0.01665,0.01665,0.01665,0.015369,0.015369,0.01665,0.015369,relax
